<a href="https://colab.research.google.com/github/InfoViz-ItalianHeritage/CHRS/blob/main/first_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def get_first_row_data_types(file_path):
    try:
        # Read only the first row of the CSV file
        df = pd.read_csv(file_path, nrows=1)
        # Get the data types of the columns
        return df.dtypes
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# --- How to use this script ---

# 1. Replace 'your_file.csv' with the actual path to your CSV file.
#    For example:
#    file_path = 'data.csv'  # if the CSV is in the same folder as your script
#    file_path = '/Users/YourName/Documents/my_data.csv' # absolute path for macOS/Linux
#    file_path = 'C:\\Users\\YourName\\Documents\\my_data.csv' # absolute path for Windows

file_path = 'ds1.csv' # <--- CHANGE THIS LINE

# 2. Call the function and print the results
data_types = get_first_row_data_types(file_path)

if data_types is not None:
    print("Data types of the first row:")
    print(data_types)

Error: The file 'ds1.csv' was not found.


First exploratory queries for the arco dataset

In [ ]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

# Replace with the ARCO / MiBACT SPARQL endpoint URL
endpoint_url = "https://dati.beniculturali.it/sparql"

query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?instance ?label
WHERE {
  ?instance a <http://dati.beniculturali.it/cis/CulturalInstituteOrSite> .
  OPTIONAL { ?instance rdfs:label ?label }
}
"""

# Set up the connection
sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

# Execute query
results = sparql.query().convert()

# Convert to pandas DataFrame for easy exploration
data = []
for result in results["results"]["bindings"]:
    organization = result.get("organization", {}).get("value", "")
    label = result.get("label", {}).get("value", "")
    data.append({"organization": organization, "label": label})

df = pd.DataFrame(data)

# Show first rows
df.head(50)


,organization,label
0,,Archivio di Stato di Firenze - Fondo Coppedè
1,,Archivio di Stato di Cagliari - Gremio dei Fal...
2,,Deposito organizzato del museo di strumentaria...
3,,Museo Archeologico Nazionale di Firenze
4,,"Museo Archeologico Nazionale ""La Civitella"""
5,,"Museo Archeologico Nazionale ""Villa Frigerj"""
6,,Museo Archeologico Nazionale di Firenze
7,,Archivio Centrale dello Stato
8,,Archivio di Stato di Agrigento
9,,Archivio di Stato di Agrigento. Sezione di Sci...


In [ ]:
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?entity ?entityLabel ?property ?propertyLabel ?value
WHERE {
  ?entity a <http://dati.beniculturali.it/cis/CulturalInstituteOrSite> .
  OPTIONAL { ?entity rdfs:label ?entityLabel }
  ?entity ?property ?value .
  OPTIONAL { ?property rdfs:label ?propertyLabel }
}
LIMIT 1000


Cultural institutions and their identifiers query

In [ ]:
PREFIX arcoLoc: <https://w3id.org/arco/ontology/location/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?entity ?label ?culturalIdentifier
WHERE {
  ?entity a <http://dati.beniculturali.it/cis/CulturalInstituteOrSite> .
  OPTIONAL { ?entity rdfs:label ?label }
  OPTIONAL { ?entity arcoLoc:culturalInstituteOrSiteIdentifier ?culturalIdentifier }
}
LIMIT 500


Query to identify classes and properties

In [ ]:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?class ?classLabel ?property ?propertyLabel ?range
WHERE {
  ?class a owl:Class .
  OPTIONAL { ?class rdfs:label ?classLabel }

  ?property rdfs:domain ?class .
  OPTIONAL { ?property rdfs:label ?propertyLabel }
  OPTIONAL { ?property rdfs:range ?range }
}
ORDER BY ?class ?property
LIMIT 1000


In [2]:
import pandas as pd
project_20 = pd.read_csv("progetti_esteso_CULTURA_TURISMO_2014-2020_20250430.csv", sep=';', low_memory=False)
project_20.head()

FileNotFoundError: [Errno 2] No such file or directory: 'progetti_esteso_CULTURA_TURISMO_2014-2020_20250430.csv'